# Домашнее задание 3

Третье домашнее задание посвящено достаточно простой, но, надеюсь, интересной задаче, в которой потребуется творчески применить методы сэмплирования. Как и раньше, в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются.

В этом небольшом домашнем задании мы попробуем улучшить метод Шерлока Холмса. Как известно, в рассказе The Adventure of the Dancing Men великий сыщик расшифровал загадочные письмена.
Пользовался он для этого так называемым частотным методом: смотрел, какие буквы чаще встречаются в зашифрованных текстах, и пытался подставить буквы в соответствии с частотной таблицей: E — самая частая и так далее.

В этом задании мы будем разрабатывать более современный и продвинутый вариант такого частотного метода. В качестве корпусов текстов для подсчётов частот можете взять что угодно, но для удобства вот вам “Война и мир” по-русски и по-английски:

In [1]:
path_to_AnnaKarenina = "data/AnnaKarenina.txt"
path_to_WarAndPeace = "data/WarAndPeace.txt"
path_to_WarAndPeaceEng = "data/WarAndPeaceEng.txt"

## Часть 1

1. Реализуйте базовый частотный метод по Шерлоку Холмсу:
    * подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
    * возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
    * расшифруйте их таким частотным методом.


In [2]:
import os
import re
import random
import string
import numpy as np
from copy import copy
from collections import Counter, defaultdict
from tqdm import tqdm



In [3]:
#support function

def read_data(path):
    with open(path, "r", encoding="utf8") as file:
        data = file.read().replace('\n', ' ')
        data = re.sub('\W+',' ', data).lower()
    return data

def show_first_elem(d, i):
    cnt = 0
    for k, v in d:
        cnt+=1
        print(f'\"{k}\" :  {v}')
        if cnt == i:
            break

In [4]:
AnnaKarenina = read_data(path_to_AnnaKarenina)
WarAndPeace = read_data(path_to_WarAndPeace)
WarAndPeaceEng = read_data(path_to_WarAndPeaceEng)

In [6]:
AnnaKarenina[:1500]

'annotation анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива по своему это книга о вечных ценностях о любви о вере о семье о человеческом достоинстве лев толстойроман широкого дыхания часть перваяi лев толстой анна каренина роман широкого дыхания анна каренина поразила современников вседневностью содержания необычайная свобода раскованность повествования удивительно сочетались в этом романе с цельностью художественного взгляда автора на жизнь он выступал здесь как художник и мыслитель и назначение искусства видел не в том чтобы неоспоримо разрешить вопрос а в том чтобы заставить любить жизнь в бесчисленных никогда не истощимых всех ее проявлениях 61 100 1 в 70 е годы один маститый писатель по видимому гончаров сказал достоевскому это вещь неслыханная это вещь первая кто у нас из писателей может поравняться с этим а в европе кто представит хоть что нибудь подоб

Возьмем из Анны Карениной тестовые тексты это будут 500, 1000 и 2000 первых символов и в качестве бонуса на всем сете.

In [7]:
def text_encoder(text):
    
    chars_in_text = list(set(text))
    
    cipher = list(set(string.printable))
    random.shuffle(cipher)
    cipher = cipher[:len(chars_in_text)]
    mapping = dict(zip(chars_in_text, cipher)) 
    ciphertext = [mapping[char] for char in text]
    
    return "".join(ciphertext)


In [8]:
text1 = AnnaKarenina[:500]
text2 = AnnaKarenina[:1000]
text3 = AnnaKarenina[:2000]

ciphertext1 = text_encoder(text1)
ciphertext2 = text_encoder(text2)
ciphertext3 = text_encoder(text3)
ciphertext4 = text_encoder(AnnaKarenina)

In [9]:
ciphertext1

'Y++@>Y>k@+b.pp.b5.tqpNp.bTVNpbN&b$.Dsfb&p.DqpN3sfbtTD.pTAbZ"A.b3TZ$3T1Tbp.zNp.q3$%b$3.A-qEb.4TtN&DTDb4t.&TEbA$qb$z.$3ZNAsqb$qD"NbMTfTLNbVt;1bp.bVt;1.b5.LV.%bpq$z.$3ZNA.%b$qD"%bpq$z.$3ZNA.bMTb$ATqD;b\n3Tb5pN1.bTbAqzpsfbgqppT$3%fbTbZiKANbTbAqtqbTb$qD"qbTbzqZTAqzq$5TDbVT$3TNp$3AqbZqAb3TZ$3TEtTD.pb-NtT5T1TbVsf.pN%bz.$3"bMqtA.%kbZqAb3TZ$3TEb.pp.b5.tqpNp.btTD.pb-NtT5T1TbVsf.pN%b.pp.b5.tqpNp.bMTt.&NZ.b$TAtqDqppN5TAbA$qVpqApT$3"ib$TVqtL.pN%bpqTKsz.Ep.%b$ATKTV.bt.$5TA.ppT$3"bMTAq$3ATA.pN%b;VNAN3qZ"pTb$Tzq'

Зашифровали, теперь надо бы получить статистику по корпусу

In [11]:
def text_letter_stats(text):
    counts = dict(Counter(text))
    counts = sorted(counts.items(), key=lambda item: item[1], reverse=True )
    counts = [(k, v/len(text)) for k, v in counts]

    return counts


In [12]:
AnnaKarenina_stats = text_letter_stats(AnnaKarenina)
show_first_elem(AnnaKarenina_stats, 10)

" " :  0.16759116836318097
"о" :  0.09513034595302555
"е" :  0.07242743491488532
"а" :  0.06859314466860275
"н" :  0.05748448067215471
"и" :  0.05498627598220739
"т" :  0.04957691600291936
"с" :  0.044003547274936064
"л" :  0.04153755858919674
"в" :  0.038988394038047686


Теперь приступим к расшифровке:

In [15]:
def text_decoder(text, global_stats):
    
    text_count = Counter(text)
    text_count = sorted(text_count.items(), key=lambda item: item[1], reverse=True)
    
    map_count = {}
    for i,value in enumerate(text_count):
        map_count[value[0]] = global_stats[i][0]
    
    
    decoded_text = []
    
    for item in text:
        decoded_text.append(map_count[item])
    
    return "".join(decoded_text)
        
    

In [16]:
text_decoder(ciphertext3, AnnaKarenina_stats)

'2ee8ъ2ъn8e аееа дарнеиеа омие иб такьж беакнеисьж рокаеол впла совтсояо еагиеанстч тсалхнз аiорибкок iрабоз лтн тгатсвильн тнкпи ыожоши мруя еа мруяа дашмач ентгатсвилач тнкпч ентгатсвила ыо тлонку юсо деияа о лнгеьж щнееотсчж о вэйли о лнрн о тнкпн о гнволнгнтдок мотсоиетслн внл совтсозрокае хиродояо мьжаеич гатсп ынрлачn внл совтсоз аееа дарнеиеа рокае хиродояо мьжаеич аееа дарнеиеа ыорабива толрнкнееидол лтнменлеотспэ томнршаеич енойьгазеач тлойома ратдолаееотсп ыолнтслолаеич умилиснвпео тогнсавитп л юсок рокаен т щнвпеотспэ жумошнтслнееояо лбявчма алсора еа шибеп ое льтсуыав бмнтп дад жумошеид и кьтвиснвп и еабеагнеин итдуттсла лимнв ен л сок гсойь енотыорико рабрнхисп лоырот а л сок гсойь батсалисп вэйисп шибеп л йнтгитвнееьж еидояма ен итсофикьж лтнж нн ырочлвнеичж 1ц цaa ц л sa н яомь омие катсисьз ыитаснвп ыо лимикоку яоегарол тдабав мотсонлтдоку юсо лнфп ентвьжаееач юсо лнфп ынрлач дсо у еат иб ыитаснвнз кошнс ыоралечсптч т юсик а л нлроын дсо ырнмтсалис жосп гсо еийумп ыомой

Собственно прочитать что то осмысленое сложно. попробуем посчитать метрики цифрами 

In [17]:
def accuracy_metric(true, pred):
    return (np.array(list(true)) == np.array(list(pred))).mean()

In [18]:
metric1 = accuracy_metric(text1, text_decoder(ciphertext1, AnnaKarenina_stats))
metric2 = accuracy_metric(text2, text_decoder(ciphertext2, AnnaKarenina_stats))
metric3 = accuracy_metric(text3, text_decoder(ciphertext3, AnnaKarenina_stats))

print(f'Метрика accuracy на выборке размером в 500: {metric1}')
print(f'Метрика accuracy на выборке размером в 1000: {metric2}')
print(f'Метрика accuracy на выборке размером в 2000: {metric3}')


Метрика accuracy на выборке размером в 500: 0.27
Метрика accuracy на выборке размером в 1000: 0.262
Метрика accuracy на выборке размером в 2000: 0.425


Метрики тоже не очень

## Часть 2

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
* подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
* проведите тестирование аналогично п.1, но при помощи биграмм.

In [19]:
def bigram_maker(text):
    bigram_text = []
    for i in  range(len(text) - 1):
        bigram_text.append(text[i:i+2])
    return bigram_text

In [20]:
def text_bigram_stats(text):
    counts = dict(Counter(bigram_maker(text)))
    counts = sorted(counts.items(), key=lambda item: item[1], reverse=True )
    counts = [(k, v/len(text)) for k, v in counts]

    return counts

In [21]:
AnnaKarenina_bigram_stats = text_bigram_stats(AnnaKarenina)
len(AnnaKarenina_bigram_stats)

1346

In [23]:
bigram_maker(AnnaKarenina)[:5]

['an', 'nn', 'no', 'ot', 'ta']

In [25]:
AnnaKarenina_bigram_stats[:8]

[('о ', 0.023729137208547666),
 ('е ', 0.018688211168292893),
 ('а ', 0.01853181711150135),
 ('и ', 0.017949000308102148),
 (' н', 0.016254438486761565),
 (' с', 0.0159967104530976),
 (' в', 0.014667653843135004),
 ('то', 0.014148683302620743)]

In [26]:
def bigram_text_decoder(text, global_bigram_stats):
    text_count = Counter(bigram_maker(text))
    text_count = sorted(text_count.items(), key=lambda item: item[1], reverse=True) 
    map_count = {}
    
    for number, (text_ngram, text_freq) in enumerate(text_count):
        for i, letter in enumerate(text_ngram):
            if letter not in map_count and global_bigram_stats[number][0][i] not in map_count.values():
                map_count[letter] = global_bigram_stats[number][0][i]
                            
    decoded_text = []

    for item in text:
        if item in map_count:
            decoded_text.append(map_count[item])
        else:
            decoded_text.append('*')

    return "".join(decoded_text)

    
    
    

In [27]:
def bigram_text_decoder(text, global_bigram_stats, global_stats):
    text_count = Counter(bigram_maker(text))
    text_count = sorted(text_count.items(), key=lambda item: item[1], reverse=True)
      
    map_count = {}
    used = set()
    to_decode = set()    
    
    
    for ecnoded_bigram, _ in text_count:
        if ecnoded_bigram[0] in map_count and ecnoded_bigram[1] in map_count:
            continue
        elif ecnoded_bigram[0] in map_count:
            mapped = map_count[ecnoded_bigram[0]]
            pos = [
                decoded_bigram for decoded_bigram, _ in global_bigram_stats 
                if decoded_bigram[0] == mapped and 
                decoded_bigram[1] not in used
            ]
            if len(pos) > 1:
                map_count[ecnoded_bigram[1]] = pos[0][1]
                used.add(pos[0][1])
            else:
                to_decode.add(ecnoded_bigram[1])
        elif ecnoded_bigram[1] in map_count:
            mapped = map_count[ecnoded_bigram[1]]
            pos = [
                decoded_bigram for decoded_bigram, _ in global_bigram_stats
                if decoded_bigram[1] == mapped and 
                decoded_bigram[0] not in used
            ]
            if len(pos) > 1:
                map_count[ecnoded_bigram[0]] = pos[0][0]
                used.add(pos[0][0])
            else:
                to_decode.add(ecnoded_bigram[0])
        else:
            pos = [
                decoded_bigram for decoded_bigram, _ in global_bigram_stats 
                if decoded_bigram[1] not in used and
                decoded_bigram[0] not in used
            ]
            map_count[ecnoded_bigram[0]] = pos[0][0]
            map_count[ecnoded_bigram[1]] = pos[0][1]
            
    text_count = [(var, cnt) for var, cnt in text_count if var in to_decode]
    global_stats = [(var, cnt) for var, cnt in global_stats if var not in used]
    
    for i, (ecnoded_bigram, _) in enumerate(text_count):
        map_count[ecnoded_bigram] = global_stats[i][0]
    
    
    decoded_text = []  
    
    for item in text:
        if item in map_count:
            decoded_text.append(map_count[item])
        else:
            decoded_text.append('*')
            

    return "".join(decoded_text)

In [28]:
decoded_bigram_text2 = bigram_text_decoder(ciphertext2, AnnaKarenina_bigram_stats, AnnaKarenina_stats)

In [29]:
decoded_bigram_text2

'1887010878 отто позитото ныот ор оо бы рто ито бы зн отнс лесо  нло ндн тоготои оа о осмия ожнзор н  жзорня сои огоо лосби ои ео вныньо ызйд то ызйдо поьыоа тиогоо лосоа ои еа тиогоо лосо вн осни й к н птодо н сигтбы биттно аы н люусо н сизи н ои еи н гилнсигиопн  ыно ното си лис  нло нязн от мознпндн ыбыотоа гоо е визсоа8 лис  нло ня отто позитото зн от мознпндн ыбыотоа отто позитото внзороло онсзи иттопнс соиытистно ею оныизьотоа тинубгоятоа оснуныо зоопнсоттно е внсио снсотоа йыосо илетн онги олоое с к н  зн оти о билетно ею ыйыньио ситтндн срдлаыо ос нзо то ьорте нт сбо йвол рыиое поп ыйыньтоп о  боло иле о тортогитои оопйоо со соыил ти с  н  г нуб тиновнзо н зорзимо е снвзно о с  н  г нуб роо осо е люуо е ьорте с уиогоолиттбы топндыо ти оо нцо бы соиы ии взнаслитоаы эх хчч х с шч и дныб ныот  оо о бя вооо иле вн соыо н й днтгознс опорол ыно нисопн й к н сице тиолбыоттоа к н сице визсоа п н й тоо ор вооо илия  ньи  внзоста еоа о к о  о с исзнви п н взиыо осо  ын е г н тоуйые вныну

In [33]:
accuracy_metric(text2, bigram_text_decoder(ciphertext2, AnnaKarenina_bigram_stats, AnnaKarenina_stats))

0.178

Стало только хуже

## Часть 3

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:

* предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
* реализуйте и протестируйте его, убедитесь, что результаты улучшились.


Что можно сделать:

1. Случайно делаем кодировщик. им восстанавливаем тект и считаем логарифм правдоподобия $p_0$.
2. меняем местами случайную пару символов в нем, так же восстанавливаем текст и считаем логарифм правдоподобия.$p_1$ 
3. с вероятностью $p = \frac{p_1}{p_0}$ берём новую перестановку.
4. Повторяем


In [34]:
def decoder_probability_calc(bigrams, decoder_map, matrix_p, letter_id_dict):
    accum  = 0
    for bigram, cnt in bigrams:
        row = letter_id_dict[decoder_map[bigram[0]]]
        col = letter_id_dict[decoder_map[bigram[1]]]
        accum += cnt * matrix_p[row, col]
    return accum 



def make_swap(decoder):
    d = copy(decoder)
    fitst, second  = np.random.choice(np.arange(len(decoder)), size=2, replace=False)
    d[fitst], d[second] = d[second], d[fitst]
    return d


def mcmc_text_decoder(text, map_count):
    decoded_text = []
    for item in text:
        decoded_text.append(map_count[item])
    
    return "".join(decoded_text)

In [35]:
def mcmc_map(encoded_text, clear_text, epoch=20, step=20000):
    
    bigrams_clear = Counter(bigram_maker(clear_text))
    bigrams_clear = sorted(bigrams_clear.items(), key=lambda item: item[1], reverse=True) 
    bigrams_encoded = Counter(bigram_maker(encoded_text))
    bigrams_encoded = sorted(bigrams_encoded.items(), key=lambda item: item[1], reverse=True)

    alphabet_clear = list(set(clear_text))
    alphabet_encoded = list(set(encoded_text))
    letter_id_dict = {char: i for i, char in enumerate(alphabet_clear)}
    
    
    matrix_p = np.ones((len(alphabet_clear), len(alphabet_clear))) 
    
    for bigram, cnt in bigrams_clear:
        row = letter_id_dict[bigram[0]]
        col = letter_id_dict[bigram[1]]
        matrix_p[row, col] += cnt
    
    matrix_p /= matrix_p.sum()
    matrix_p = np.log(matrix_p)
    
    probabilities = []
    mapps = []
    
    for _ in tqdm(range(epoch), position=0, total=epoch):
        alphabet_clear_copy = copy(alphabet_clear)
        decoder_map = dict(zip(alphabet_encoded, alphabet_clear))
        p0 = decoder_probability_calc(bigrams_encoded, decoder_map, matrix_p, letter_id_dict)
        
        for i in range(step):
            alphabet_clear_swapped = make_swap(alphabet_clear_copy)
            decoder_map_swapped = dict(zip(alphabet_encoded, alphabet_clear_swapped))
            p1 = decoder_probability_calc(bigrams_encoded, decoder_map_swapped, matrix_p, letter_id_dict)
            if p1 > p0:
                alphabet_clear_copy = alphabet_clear_swapped
                p0 = p1
                decoder_map = decoder_map_swapped
            elif np.exp(p1 - p0) > np.random.rand():
                alphabet_clear_copy = alphabet_clear_swapped
                p0 = p1
                decoder_map = decoder_map_swapped        
        probabilities.append(p0)
        mapps.append(decoder_map)
    
    
    top_idx = np.argmax(probabilities)   
    return mapps[top_idx]

In [36]:
map1 = mcmc_map(ciphertext1, AnnaKarenina)
map2 = mcmc_map(ciphertext2, AnnaKarenina)
map3 = mcmc_map(ciphertext3, AnnaKarenina)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [02:26<00:00,  7.33s/it]


In [37]:
decoded1 = mcmc_text_decoder(ciphertext1, map1)
decoded2 = mcmc_text_decoder(ciphertext2, map2)
decoded3 = mcmc_text_decoder(ciphertext3, map3)

In [38]:
decoded2

'aiiemamtei анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива по своему это книга о вечных ценностях о любви о вере о семье о человеческом достоинстве лев толстойроман широкого дыхания часть перваяt лев толстой анна каренина роман широкого дыхания анна каренина поразила современников вседневностью содержания необычайная свобода раскованность повествования удивительно сочетались в этом романе с цельностью художественного взгляда автора на жизнь он выступал здесь как художник и мыслитель и назначение искусства видел не в том чтобы неоспоримо разрешить вопрос а в том чтобы заставить любить жизнь в бесчисленных никогда не истощимых всех ее проявлениях l1 188 1 в 38 е годы один маститый писатель по видимому гончаров сказал достоевскому это вещь неслыханная это вещь первая кто у нас из писателей может поравняться с этим а в европе кто представит хоть что нибудь подоб

In [39]:
accuracy_metric(text1, decoded1)

0.852

In [40]:
accuracy_metric(text2, decoded2)

0.986

In [41]:
accuracy_metric(text3, decoded3)

0.991

Получился отличный результат!

## Часть 4

Расшифруйте сообщение:

In [42]:
message = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"

In [43]:
message_map = mcmc_map(message, AnnaKarenina)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [01:10<00:00,  3.54s/it]


In [44]:
decoded_message = mcmc_text_decoder(message, message_map)

In [45]:
decoded_message

'если ву вимите норкальную или почти норкальную тедст ы этого сообшения доторую легдо прочитать сдорее всего ву все смелали правильно и полычите кадсикальную балл за послемнее четвертое замание дырса хотя донечно я ничего не обешаф'

Ура! Получилось! (^_^)